# Demographic implications of cloning in marine invertebrate larvae 
This is a graphical interface for using the `clone_modelND` (for Non-Dimensional) model of cloning in marine invertebrate larvae. The `clone_modelND` model calculates the size distribution of a cohort of marine invertebrate larvae over time. This cohort represents a group of larvae beginning development at the same time and at the same size. For example, the cohort could represent larvae from the eggs of a single mother, or larvae from the eggs of a local population of mothers that spawn synchronously at a given time and place.

In "normal" development (without cloning), a cohort of larvae always decreases in population due to mortality, because there is no mechanism for adding new larvae to the cohort. In a cloning species, one larva can divide into two genetically identical "daughter" larvae. This means that, in cloning larvae, the number of larvae in a cohort can either increase or decrease at any given time during development.  

Like most larval life history models in the literature, **size is assumed in the** `clone_modelND` **model to be a *proxy* for developmental state**. This means that the model does not represent developmental state directly. Instead, it assumes that developmental state is closely coupled to size. For example, a larva is assumed to be competent to settle when it reaches a specific size. This approach means that the model does not exactly describe many details of morphology and life history associated with any given species. However, the more general results of this modeling approach may approximately describe a great many different species.

While cloning produces two larvae out of one, it does not provide any additional tissue or nutritional resources to those larvae. This means a larva that clones produces two daughter larvae that are smaller. The model assumes that the daughter larvae divide the resources from the original larva. Hence, **the sizes of two daughters produced by cloning add up to the size of the original larva.**

For example, some marine invertebrate larvae simply bifurcate during development, dividing their tissue into two parts. Each of these parts can either redifferentiate into a complete, smaller larva, or resume growing to reform the lost body parts. Whether it redifferentiates or regrows lost parts, a clone has to grow back to a size at which it is capable of metamorphosis (or of cloning again). The model neglects this difference in developmental mode of clones: Consistent with using size as a proxy, the model treats all larvae of size $s$ the same. 

The model can simulate characteristics of two distinct modes of cloning, exemplified by cloning typically found in Echinoids and Ophiuroids:

- In the **Echinoid mode** of cloning, larvae grow and develop to a threshold size smaller than the size at metamorphosis. At this threshold size, some fraction of individuals clone to form two smaller daughters. These clones resume growth, and either clone again when they reach the threshold size or proceed to metamorphosis. Clones resume growth, and either clone again when they reach the threshold size or proceed to metamorphosis.

- In the **Ophiuroid mode** of cloning, a fraction of surviving larvae grow past the size at metamorphosis. These larger larvae then divide into two clones, one of which metamorphoses immediately. The other, smaller clone then resumes development, repeating growth and either metamorphosing directly or cloning again to form a metamorph and yet another small clone.

The `clone_modelND` model is capable of modeling many scenarios, such as fluctuating environments or multiple potential cloning or metamorphosis sizes, that are not implemented through this Jupyter NoteBook interface. These can be accessed by running the model directly from Python.

## Model structure and parameters
The `clone_modelND` model has two independent variables: larval size, $s$, and time, $t$. It has one dependent variable: $p(s,t)$, the population of larvae of size $s$ at time $t$. 

The model tracks how $p(s,t)$ evolves as a function of time, under the influence of four mechanisms:

- **Larval growth and development**
    - $g_0$ determines the basal growth rate.
 
- **Larval mortality**
    - $m_0$ determines the basal mortality rate.
    - $\alpha$ determines the *size-dependence* of mortality, e.g. due to predators that preferentially prey on early or late stage larvae ($\alpha<0$ means mortality is higher for early stages; $\alpha>0$ means mortality is higher for late stages; $\alpha=0$ means mortality is the same for all stages).  
    *Note:* These parameters interact so changing $\alpha=0$ also affects the impact of $m_0$. 
     
- **Sizes for developmental milestones**
    - $s_{egg}$ determines egg size (starting size for a developing larva)
    - $s_0$ determines the size at cloning
    - $s_1$ determines the size of one clone (must have $s_1<s_0$). The other clone has size $s_0-s_1$.
    - $s_2$ is the size at metamorphosis
    - $s_{max}$ determines the upper limit of the population size structure plot. It has no effect on the simulation itself.
     
- **Probabilities for developmental milestones**
    - $c_0$ is the probability a larva will clone when it reaches size $s_1$. $c_0=0$ means larvae never clone. $c_0 = 0.5$ means 50% of larvae clone, etc.
    - $f_2$ is the probability of metamorphosis at size $s_2$. $f_2=1$ means larvae always metamorphose when they reach size at metamorphosis. $f_2 = 0.5$ means 50% of larvae metamorphose when they reach size at metamorphosis.

The *initial condition* -- the size distribution at the beginning of a simulation -- is that all larvae are initially of size $s_0$. Because there are only a fixed amount of maternal resources to put into eggs, making larger eggs means there are initially fewer, larger larvae. Conversely, making smaller eggs means there are initially more, smaller larvae.

## Meaningful parameter choices
The "ND" in the name of the `clone_modelND` model indicates that the model is **Non-Dimensional**.
This means key quantities have been scaled by some parameters, to remove redundant axes of variation.
- The population of larvae have been normalized by the total amount of maternal resources. If $N$ is the initial number of larvae, $ N \times S_0 = 1$. 
- The sizes of larvae have been normalized by the size at metamorphosis. This means that you should keep $s_2=1$.
- The timescale has been scaled so that $g_0 = 1$. This means, essentially, that if larvae take approximately a week to develop, we count time in weeks. If larvae take a month to develop, we count time in months, etc.

The mortality parameters $m_0$ and $\alpha$ reflect the size and time scales described above. For example, a well developed late stage larva has scaled size approaching $s=1$, while a very early stage larva is close to $s=0$. Also, the mortality rate is relative to the time scale of growth. That is, if growth take approximately a week to develop, the mortality rate is per week, etc. 

## Modes of cloning
- To simulate the **Echinoid mode** of cloning, set:

    $f_2=1$
    
    $s_1<s_2$
    
    $0 \le c_0 < 1$

- To simulate the **Ophiuroid mode** of cloning:

    $f_2<1$
    
    $s_1>s_2$
    
    $c_0 = 1$


## Understanding model output
The output of this graphical interface for the `clone_modelND` model comprises three plots.

- The plot at the left is a **heat map of size distribution of larvae, $p(s,t)$, over larval size ($s$) and time ($t$)**. A vertical line represents the size distribution at one point in simulated time. A horizontal line represents the abundance of larvae of a given size across time.

- The plot at the upper right shows the cumulative number of metamorphs, $F(t)$, across time. The cumulative number of metamorphs at the end of the simulation is given just above the end of this line.

- The plot at the lower right shows the total number of developing larvae of all sizes, $P(t)$, across time. The total number of larvae still developing at the end of the simulation is given just above the end of this line.


In [4]:
# Load modules and set graphics environment
from math import *
import numpy as np
from matplotlib import pyplot as plt
plt.ion();
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from Cloning.clone_modelND import Params, ClonePDE

In [6]:
# Instantiate a Params object and a GUI to modify parameters. 
params=Params(auto_plot=True)
# Restrict to a single environment, to simplify the interface
#save_all=False,s_egg=0.1,s_0=0.75,s_1=0.25,s_2=1.,
#                 Smax=1.25,s_j=1.,N0=1.,n_env=1,
#                 t_env_cycle=1.,env_cycle_ratios=np.array([1.]),g_pars=[{'g0':1.}],
#                 m_pars=[{'m0':0.15}],alpha=-0.75,c_0 = 0.5,f_2 = 1.,C = None,
#                 nruns=1,start_time=0.,end_time=10.,dt=0.05,max_step=0.005,ns=4*32,
#                 abserr=1.e-12,relerr=1.e-10,auto_plot=False
s_egg=widgets.FloatText(value=0.1,width=10,description = r"$s_{egg}$")
s_0=widgets.FloatText(value=0.75,description = r"$s_0$")
s_1=widgets.FloatText(value=0.25,description = r"$s_1$")
s_2=widgets.FloatText(value=1.0,description = r"$s_2$")
Smax=widgets.FloatText(value=1.25,description = r"$s_{max}$")
Tmax=widgets.FloatText(value=10.,description = r"$T_{max}$")
g0=widgets.FloatText(value=1.,description = r"$g_0$")
m0=widgets.FloatText(value=0.,description = r"$m_0$")
alpha=widgets.FloatText(value=-0.75,description = r"$\alpha$")
c_0=widgets.FloatText(value=0.,description = r"$c_0$")
f_2=widgets.FloatText(value=1.,description = r"$f_2$")

h = '30px'
L_s_egg = widgets.Label(value='Egg size',layout=widgets.Layout(display="flex", 
                                        justify_content="flex-end",width="97%",height=h))
L_s_0 = widgets.Label(value='Size at cloning',layout=widgets.Layout(display="flex", 
                                        justify_content="flex-end",width="95%",height=h))
L_s_1 = widgets.Label(value='Clone size',layout=widgets.Layout(display="flex", 
                                        justify_content="flex-end",width="95%",height=h))
L_s_2 = widgets.Label(value='Size at metamorphosis',layout=widgets.Layout(display="flex", 
                                        justify_content="flex-end",width="97%",height=h))
L_Smax = widgets.Label(value='Plot limit',layout=widgets.Layout(display="flex", 
                                        justify_content="flex-end",width="90%",height=h))
uiL1 = widgets.VBox([L_s_egg,L_s_0,L_s_1,L_s_2,L_Smax])

L_g0 = widgets.Label(value='Growth rate',layout=widgets.Layout(display="flex", 
                                        justify_content="flex-start",width="97%",height=h))
L_m0 = widgets.Label(value='Mortality rate',layout=widgets.Layout(display="flex", 
                                        justify_content="flex-start",width="95%",height=h))
L_alpha = widgets.Label(value='Mort. selectivity',layout=widgets.Layout(display="flex", 
                                        justify_content="flex-start",width="95%",height=h))
L_c_0 = widgets.Label(value='Cloning probability',layout=widgets.Layout(display="flex", 
                                        justify_content="flex-start",width="95%",height=h))
L_f_2 = widgets.Label(value='Metamorphosis probability',layout=widgets.Layout(display="flex", 
                                        justify_content="flex-start",width="97%",height=h))
L_Tmax = widgets.Label(value='End time',layout=widgets.Layout(display="flex", 
                                        justify_content="flex-start",width="90%",height=h))
uiL2 = widgets.VBox([L_g0,L_m0,L_alpha,L_c_0,L_f_2,L_Tmax])


LabelObj = widgets.Label(value='Object',layout=widgets.Layout(display="flex", 
                                        justify_content="center",width="100%"))
LabelObj1 = widgets.Label(value='Object')
Space = widgets.Label(value=' ',
                         layout=widgets.Layout(display="flex", 
                                               justify_content="center", 
                                               width="100%"))
LabelModel = widgets.Label(value='Model',
                         layout=widgets.Layout(display="flex", 
                                               justify_content="flex-end", 
                                               width="150%"))
ui0 = widgets.VBox([LabelObj,LabelObj1,Space,LabelModel])
ui1 = widgets.VBox([s_egg,s_0,s_1,s_2,Smax])
ui2 = widgets.VBox([g0,m0,alpha,c_0,f_2,Tmax])
ui3 = widgets.HBox([uiL1,ui1,ui2,uiL2])

def setParams(s_egg,s_0,s_1,s_2,Smax,g0,m0,alpha,c_0,f_2,Tmax):
    params=Params(auto_plot=True)
    params.s_egg = s_egg
    params.s_0 = s_0
    params.s_1 = s_1
    params.s_2 = s_2
    params.Smax = Smax
    params.g_pars = [{'g0':g0}]
    params.m_pars = [{'m0':m0}]
    params.alpha = alpha
    params.c_0 = c_0
    params.f_2 = f_2
    params.end_time = Tmax
    result=ClonePDE(params)
    
out = widgets.interactive_output(setParams,{'s_egg':s_egg,'s_0':s_0,'s_1':s_1,'s_2':s_2,'Smax':Smax,
                                             'g0':g0,'m0':m0,'alpha':alpha,'c_0':c_0,'f_2':f_2,'Tmax':Tmax})
display(ui3,out)

Output()